# Segmenting and Clustering Neighborhoods in Toronto

## Applied Data Science Capstone, Week 3

## Peer-Graded Assignment

### 1. Creating the Toronto dataframe

#### Let's start by downloading all of our dependencies

In [1]:
import numpy as np
import pandas as pd
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

We're going to need <b>BeautifulSoup 4</b> for scraping the info from Wikipedia. Let's load it...

In [2]:
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

postal_page=requests.get(url)

postal_page.content

soup = BeautifulSoup(postal_page.content, 'html.parser')

#### Now we can finally create our Toronto dataframe

In [4]:
#using pandas to read the table into a dataframe
toronto_table = soup.find('table')
df = pd.read_html(str(toronto_table))

toronto_df = df[0]
toronto_df.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)
toronto_df.head()

#another approach, which relies more on BeautifulSoup, to create the postal code df is below
    #toronto_table = []
    #table_rows = toronto_table_elem.find_all('tr')
        #for tr in table_rows:
            #td = tr.find_all('td')
            #row = [tr.text for tr in td]
            #toronto_table.append(row)
    #toronto_df = pd.DataFrame(toronto_table, columns=["PostalCode", "Borough", "Neighborhood"])

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ok, looking good so far! 
Now we need to <b>drop</b> the cells where <b>Borough = Not assigned</b>, and we need to <i>update</i> any rows where Neighborhood = Not Assigned so that <i>Neighborhood = Borough</i>.

In [5]:
toronto_updated = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_updated.loc[toronto_updated['Neighborhood'] == 'Not assigned', ['Neighborhood']] = toronto_updated['Borough']
toronto_updated

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


I saved the best for last... maybe? Not really, this was just the most challenging part of the problem for me. <br>
Now it's time to tackle the following: <br>
<i>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</i>

In [6]:
toronto_final = toronto_updated.groupby('PostalCode', as_index=False).agg(lambda x:', '.join(set(x)))

toronto_final

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### The final part of this section is to print the shape of this dataframe, which is below

In [7]:
toronto_final.shape

(103, 3)

#### 2. Adding geospatial data to Toronto dataframe

In [8]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
geo_data.rename(columns={"Postal Code": "PostalCode"}, inplace = True)

In [10]:
toronto_final['Latitude']=geo_data['Latitude'].values
toronto_final['Longitude']=geo_data['Longitude'].values

toronto_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### 3. Explore and cluster Toronto neighborhoods

Let's get the geographical (lat, long) coordinates of Toronto, Canada.

In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Like with the example of NY in the previous lab, let's create a map of Toronto with its neighborhoods superimposed over it.

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for postcode, lat, lng, borough, neighborhood in zip(toronto_final['PostalCode'], toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Borough'], toronto_final['Neighborhood']):
    label = '{}, {}, {}'.format(postcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

As per the suggestion in the instructions for this assignment, I will be working only with boroughs that contain the word Toronto to replicate the same analysis done with the NYC data. Let's slice the <i>toronto_final</i> dataframe and create the narrowed down dataframe.

In [14]:
toronto_data = toronto_final[toronto_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
#creating map of only boroughs that contain Toronto
map_torbor = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for postcode, lat, lng, neighborhood in zip(toronto_data['PostalCode'], toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(postcode, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torbor)  
    
map_torbor

We're ready to get to the fun part now! Let's use the FourSquare API to explore and segment.

In [17]:
#Let's define all the variables needed to create the URL for the API call

CLIENT_ID = '2W2BM5Q5KFXA1KYRKK2F3H3I413J5RD2LHVDA2ZWFNI24Z5T' # my Foursquare ID
CLIENT_SECRET = 'QTFPM40TQ5UY0HMKTDJJ22F21DUBHQT4ONLT0Q51G44AU1O5' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

I'm going to borrow the function from the lab that fetches the top 100 venues within a radius of 500 meters that are in each neighborhood in the toronto_data dataframe.

In [25]:
def getNearbyVenues(postalcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postalcode, lat, lng in zip(postalcodes, latitudes, longitudes):
        print(postalcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's run the above function on each postal code and create a new dataframe called *torbor_venues*. Let's check its size and see the first 5 rows.

In [26]:
torbor_venues = getNearbyVenues(postalcodes=toronto_data['PostalCode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(torbor_venues.shape)
torbor_venues.head()

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7A
M7Y
(1714, 7)


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's examine how many venues were returned for each neighborhood.

In [28]:
torbor_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,5,5,5,5,5,5
M4K,42,42,42,42,42,42
M4L,20,20,20,20,20,20
M4M,42,42,42,42,42,42
M4N,4,4,4,4,4,4
M4P,7,7,7,7,7,7
M4R,22,22,22,22,22,22
M4S,33,33,33,33,33,33
M4T,1,1,1,1,1,1


And now let's see how many unique categories exist.

In [29]:
print('There are {} uniques categories.'.format(len(torbor_venues['Venue Category'].unique())))

There are 236 uniques categories.


We'll need to perform one-hot encoding to analyze and cluster our selected neighborhood.

In [31]:
# one hot encoding
torbor_onehot = pd.get_dummies(torbor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torbor_onehot['Postal Code'] = torbor_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [torbor_onehot.columns[-1]] + list(torbor_onehot.columns[:-1])
torbor_onehot = torbor_onehot[fixed_columns]

torbor_onehot.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category as was shown in the lab.

In [32]:
torbor_grouped = torbor_onehot.groupby('Postal Code').mean().reset_index()
torbor_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,...,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.023810
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.023810,0.000000,0.00,0.023810
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.045455
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.000000


Let's print each postal code along with the top 5 most common venues.

In [33]:
num_top_venues = 5

for code in torbor_grouped['Postal Code']:
    print("----"+code+"----")
    temp = torbor_grouped[torbor_grouped['Postal Code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                  venue  freq
0  Other Great Outdoors   0.2
1                   Pub   0.2
2     Health Food Store   0.2
3          Neighborhood   0.2
4                 Trail   0.2


----M4K----
                    venue  freq
0        Greek Restaurant  0.21
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4          Ice Cream Shop  0.05


----M4L----
            venue  freq
0            Park  0.10
1  Sandwich Place  0.10
2     Pizza Place  0.05
3     Coffee Shop  0.05
4         Brewery  0.05


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----M4N----
                venue  freq
0  Photography Studio  0.25
1                Park  0.25
2            Bus Line  0.25
3         Swim School  0.25
4   Afghan Restaurant  0.00


----M4P----
              venue  freq
0    Breakfast Spot  0.14
1       

And of course we need to put this into a pandas df!<br>
Let's start by sorting the venues into descending order, so we can set up our df to show the 10 most common venue types in each postal code.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcodes_venues_sorted = pd.DataFrame(columns=columns)
postalcodes_venues_sorted['Postal Code'] = torbor_grouped['Postal Code']

for ind in np.arange(torbor_grouped.shape[0]):
    postalcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torbor_grouped.iloc[ind, :], num_top_venues)

postalcodes_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Other Great Outdoors,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Dessert Shop
2,M4L,Park,Sandwich Place,Gym,Movie Theater,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place
4,M4N,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Now we'll run some K Means Clustering on this data and segment the postal codes into 5 clusters.

In [59]:
# set number of clusters
kclusters = 7

#create sub df for clustering
torbor_grouped_clustering = torbor_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torbor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 1, 1, 1, 4, 1, 1, 1, 2, 1], dtype=int32)

Let's create a new dataframe that includes the clusters as well as the top 10 venues for each postal code.

In [60]:
# add clustering labels
postalcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [61]:
torbor_merged = toronto_data

# merge dfs to add latitude/longitude for each postal code
torbor_merged = torbor_merged.join(postalcodes_venues_sorted.set_index('Postal Code'), on='PostalCode')

torbor_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,5,Neighborhood,Other Great Outdoors,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Dessert Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Sandwich Place,Gym,Movie Theater,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Let's map our clusters for visual insight!

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torbor_merged['Latitude'], torbor_merged['Longitude'], torbor_merged['PostalCode'], torbor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Just by looking at the map, it seems rather curious that almost all of the postal codes fell into Cluster 1. Let's explore each cluster to see what sets them apart.

<b>Cluster 0</b>

In [80]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 0, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,0,Trail,Jewelry Store,Park,Sushi Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


<b>Cluster 1</b>

In [81]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 1, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Dessert Shop
2,M4L,1,Park,Sandwich Place,Gym,Movie Theater,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,1,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place
5,M4P,1,Gym,Hotel,Park,Breakfast Spot,Department Store,Sandwich Place,Food & Drink Shop,Distribution Center,Dog Run,Donut Shop
6,M4R,1,Clothing Store,Coffee Shop,Yoga Studio,Dessert Shop,Restaurant,Mexican Restaurant,Sporting Goods Shop,Chinese Restaurant,Fast Food Restaurant,Spa
7,M4S,1,Dessert Shop,Pizza Place,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Brewery,Gas Station
9,M4V,1,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
11,M4X,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Pub,Bakery,Pizza Place,Pet Store,Park,Pharmacy
12,M4Y,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gastropub
13,M5A,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Theater,Restaurant,Café,Mexican Restaurant,Chocolate Shop


<b>Cluster 2</b>

In [82]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 2, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


<b>Cluster 3</b>

In [83]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 3, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,3,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<b>Cluster 4</b>

In [84]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 4, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,4,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<b>Cluster 5</b>

In [85]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 5, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,5,Neighborhood,Other Great Outdoors,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store


<b>Cluster 6</b>

In [86]:
torbor_merged.loc[torbor_merged['Cluster Labels'] == 6, torbor_merged.columns[[0] + list(range(5, torbor_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M4W,6,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## CONCLUSION: It seems the density of Cluster 1 can be explained by the fact that the postal codes mostly have coffee shops and cafes as their top venue(s). The rest of the clusters each have distinct distributions of top venues.